# 广告技术篇
## 程序化广告基本流程

### 程序化广告过程示意图

![程序化广告过程示意图](../imgs/程序化广告过程示意图.png)

上图展示了程序化广告从用户访问APP触发一次广告机会到最后广告成功展示的流程示意图。
#### 1. 广告请求过程

步骤1.1：用户浏览媒体APP，触发一次广告曝光机会。

步骤1.2：媒体APP通过SDK向SSP发起广告请求，并附带广告位，设备信息，上下文等数据。这些信息原则上会在整请求链路中一直带下去。

步骤1.3：SSP会向各家ADX发起广告请求，并添加底价，DealId信息。这里假设ADX1为我们的FTX系统，便于大家可以对应了解。

步骤1.4：收到广告请求的ADX会根据平台配置向各家符合条件的DSP发起竞价请求（**不会向所有已对接的DSP发起询价请求**），并同时附带得到的广告位，设备信息，上下文，底价，DealId等信息。这里假设DSP1为我们的FancyDSP系统。

#### 2. 广告响应过程

步骤2.1：各家DSP根据收到的广告竞价请求及附带的信息，决定1. 是否参加 2. 展示哪个广告 3. 参竞出价，并把最后的参竞信息响应给ADX。参竞信息包含匹配的广告创意地址，各方数据监测地址，以及出价信息。

步骤2.2：ADX收到各家DSP的响应信息后，提取参竞的DSP的信息，并选出胜出的DSP，并把胜出的广告信息返回给SSP，附带自家的数据监测地址。一般是高价者得，当然还有其它一些规则限制，比如频控，PD，PDB优先等。

步骤2.3：SSP收到各家ADX的响应信息收，类似ADX一样，从参竞的ADX的信息中，选出胜出的ADX，并把广告信息返回给媒体APP，附带自家的数据监测地址。

#### 3. 展示阶段

步骤3.1：媒体APP在收到SSP的广告展示信息后，开始把广告创意渲染出来展示给用户。
- 同时请求拿到的曝光监测地址实现曝光数据上报。
- 假如用户此时发生了点击，那么媒体APP则会请求点击监测地址实现点击数据上报。

### 程序化广告流量时序图
由于各家SSP，ADX，DSP的接口协议大同小异，实际上ADX在转发广告请求，选择胜出广告的同时，还有一个更重要的职责，那就是适配。
- 把各家SSP，ADX的请求数据适配成各家DSP接口支持的数据格式。
- 把各家DSP的响应数据是配成各家SSP，ADX接口支持的数据格式。

![程序化广告流量时序图](../imgs/程序化广告流量时序图.png)

上图中的的整体流程与程序化广告过程示意图接近，不再缀述，核心描述下几个重要步骤。可参考ods.o_ad_ftx_raw_log中的抽样数据，加深理解。

- 2. bid_data：媒体端发送给FTX的原始请求数据。
- 3. dsp_request_data： FTX适配后发送给DSP的请求数据。
- 4. dsp_response_data：DSP响应给FTX的原始数据。
- 5. response_data：FTX适配以后发送给媒体的响应数据。
- 上述4个数据，粗看这里并没有什么，实际上即有修改，就有出错的可能性，在一些问题排查过程中，往往就在做 bid_data vs dsp_request_data，dsp_response_data vs response_data中得到答案。
- 7. 曝光监测回调：FTX与FancyDSP共用一套曝光监测
- 8. 点击监测回调：FTX与FancyDSP有各自的点击监测地址。并且大部分媒体上，FancyDSP用的是同步点击监测路径，FTX用的异步点击监测路径。往往配合排查可以解决很多点击GAP问题。参见[数据监测篇#监测上报流程](./数据监测篇.ipynb#监测上报流程)了解更多的信息。



## 程序化广告技术指标
### 原始统计指标
- 请求量：Vendor发送给FTX的请求数。对应步骤1.3
    + 指标出现在媒体广播量报表中，一次媒体请求只计一次。
- 可发送PV：Vendor发送给FTX的请求数。对应步骤1.3
    + 在FTX多维度报表中此数会根据实际发送给DSP的情况，同比放大。
- 实际发送PV：FTX发送给具体DSP的请求数。对应步骤1.4。这个指标是面向DSP角度的，它有以下几个特征。
    + 小于等于可发送PV。上述说过，并不是所有媒体请求都会转发给某个DSP，由运营执行策略控制。
    + 求和值会大于媒体发过来的去重请求数。由于一次媒体请求，会被同时发送给多个DSP，每个DSP都会记录一次实际发送PV，于是求和值会出现大于媒体发送过来的去重请求数。
- 响应成功PV：DSP成功返回并且参竞的响应数。对应步骤2.1。这个指标依旧是面向DSP角度的，它有以下几个特征。
    + 小于等于实际发送PV。只有发送给DSP了，才会有响应。
    + 求和值与媒体发过来的去重请求数无可比性。原因同上。
- 内部竞价数：FTX内部选中并响应给Vendor的参竞数。对应步骤2.2。这个指标是面向Vendor角度了，它有以下几个特征。
    + 小于等于媒体发过来的去重请求数。由于一次媒体请求，无论多少个DSP参竞，最后最多只能选出一个胜出DSP，转发给Vendor进行参竞。
    + 内部参竞不代表一定可以被Vendor选中。
    + **在媒体广播量报表中，取名参竞量，到这一步开始，每一次媒体请求，至多对应一条数**
- 媒体胜出数：标准定义FTX的参竞广告在Vendor服务端的胜出数。对应步骤2.3中返回是FTX的参竞广告数。
    + 实际上只有QTT给我们返回了标准的媒体胜出数。
    + 其它媒体返回的都是广告在媒体APP端的实际曝光数。等同展现量
- （总）展现量：FTX的广告在媒体App端的成功展现数，也就是广告考核的曝光数。对应曝光监测回调的数。
    + 总展现量：曝光监测收到到原始展现数。
    + 展现量：原始展现数中请求ID的去重数。
- （有效）点击量：FTX的广告在媒体APP端的用户点击数。也就是广告考核的点击数。对应点击监测回调的数。
    + 总点击量：点击监测收到的原始点击数。
    + 有效点击量：原始点击数经过GIVT后的有效点击数。去重点击只是其中一个规则。
- 吊起数：FTX的广告在媒体APP端的用户吊起（唤起）数。对应吊起监测回调的数。
    + 不是所有媒体都支持吊起回调，只有QTT，韩剧TV，人人视频等几家媒体支持。
    + 这里的吊起数并不是APP真实成功唤起数，因为媒体会在尝试唤起APP之前回调监测地址。

### 计算指标
- (DSP)参与竞价率 := (响应成功PV)/(实际发送PV) * 100%
    + 主要用与评估DSP在具体媒体上的参竞情况。
- (Vendor)参竞率 := (参竞量)/(请求量) * 100%
    + 评估FTX对某个具体媒体请求的真实参竞情况，无关具体的DSP。
- 内部竞得率 := (内部竞价数)/(响应成功PV) * 100%
    + 评估具体DSP参竞后在FTX内部的胜出情况。假如内部竞得率一致很低，则可能运营策略上出问题了，或者被其它更高优先级的订单抢量了。
- 媒体胜出率 := (媒体胜出数)/(内部竞价数) * 100%
    + 评估FTX的参竞在具体媒体上的胜出情况，实际上只针对QTT有效，其它媒体等同内部曝光率。
- 媒体曝光率 := (展现量)/(媒体胜出数) * 100%
    + 评估媒体客户端的展现成功率，只针对QTT有效，其它媒体此值约等于100%。
- 内部曝光率 := (展现量)/(内部竞价率) * 100% := 媒体曝光率 * 媒体胜出率 * 100%
    + 评估FTX的参竞广告在媒体客户端的成功曝光率。从公式可以看出，当值较低时，我们无法知道是在媒体服务端胜出率低，还是媒体广告位的曝光率低。
    + 不过在一个稳定的媒体上，我们一认为固定的广告位的真实媒体曝光率是稳定的，从而推导出，具体广告位的内部曝光率波动可理解为媒体服务端的胜出率波动。
- 有效点击率 := (有效点击数)/(展现量) * 100%
- 吊起率 := (吊起数)/(有效点击数) * 100%
    + 这里有个问题，那就是吊起数只是简单去重，而有效点击则是GIVT过滤后的，偶尔会出现吊起率>100%的情况。
    

